In [ ]:
import numpy as np
import porepy as pp
import scipy.sparse as sps
import scipy
import sys

/usr/local/lib/python3.10/dist-packages/porepy/numerics/nonlinear/nonlinear_solvers.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange  # type: ignore


In [ ]:
main_folder = "./"
spe10_folder = main_folder + "spe10"
sys.path.insert(1, spe10_folder)

from functions import *
from spe10 import Spe10

In [ ]:
def upscale(sd, perm, dir, export_folder=None):
    """
    Compute the averaged gradient and flux for a given subdomain and direction of the pressure
    gradient.

    Args:
        sd (pp.Grid): The grid representing the subdomain.
        perm (dict): The permeability of the subdomain divided in the fields "kxx" and "kyy"
        dir (int): The direction of the flow, 0 means x-direction and 1 means y-direction.
        export_folder (str): If given, path where to export the results.
            Default to None, no exporting.

    Returns:
        (np.ndarray, np.ndarray): averaged gradient and flux.
    """
    # Permeability
    perm_tensor = pp.SecondOrderTensor(kxx=perm["kxx"], kyy=perm["kyy"])

    # Boundary conditions
    b_faces = sd.tags["domain_boundary_faces"].nonzero()[0]
    b_face_centers = sd.face_centers[:, b_faces]

    # Find the min and max values of the boundary faces
    sd_min = np.amin(sd.face_centers[dir, :])
    sd_max = np.amax(sd.face_centers[dir, :])

    # define outflow and inflow type boundary conditions
    out_flow = np.isclose(b_face_centers[dir, :], sd_max)
    in_flow = np.isclose(b_face_centers[dir, :], sd_min)

    # define the labels and values for the boundary faces
    labels = np.array(["neu"] * b_faces.size)
    labels[np.logical_or(in_flow, out_flow)] = "dir"

    bc_val = np.zeros(sd.num_faces)
    bc_val[b_faces[in_flow]] = sd_max - sd_min

    bc = pp.BoundaryCondition(sd, b_faces, labels)

    # Collect all parameters in a dictionary
    key = "flow"
    parameters = {"second_order_tensor": perm_tensor, "bc": bc, "bc_values": bc_val}
    data = pp.initialize_default_data(sd, {}, key, parameters)

    # Discretize the problem (construct the lhr and rhs)
    #discr = TODO
    #discr.discretize TODO
    discr = pp.Mpfa(key)
    discr.discretize(sd, data)

    #A, b = TODO
    A, b = discr.assemble_matrix_rhs(sd, data)

    # Solve the linear system and compute the pressure
    # p = TODO
    p = sps.linalg.spsolve(A, b)

    # Export the solution
    if export_folder is not None:
        save = pp.Exporter(sd, "sol", folder_name=export_folder)
        save.write_vtu([("p", p), ("log_perm", np.log10(perm["kxx"]))])

    # Post-process the solution to get the flux
    return compute_avg_q_grad(sd, p, data, key, bc, bc_val)

In [ ]:
def compute_tensor(grad_h, grad_v, q_h, q_v):
    """
    Compute the upscaled permeability tensor.

    Args:
        grad_h (np.ndarray): Gradient in the horizontal direction.
        grad_v (np.ndarray): Gradient in the vertical direction.
        q_h (np.ndarray): Flux in the horizontal direction.
        q_v (np.ndarray): Flux in the vertical direction.

    Returns:
        perm (np.ndarray): Upscaled permeability tensor.

    The function solves a linear system to obtain the upscaled permeability tensor
    based on the given gradients and fluxes. It enforces numerical symmetry and
    checks if the resulting tensor is symmetric positive definite (SPD).
    """
    # Solve the linear system to get the upscaled permeability
    # TODO
    lhs = np.array([
        [grad_h[0], grad_h[1], 0, 0],
        [0, 0, grad_h[0], grad_h[1]],
        [grad_v[0], grad_v[1], 0, 0],
        [0, 0, grad_v[0], grad_v[1]],
        [0, 1, -1, 0]
    ])

    rhs = np.array([q_h[0], q_h[1], q_v[0], q_v[1], 0])

    perm = np.linalg.lstsq(lhs, rhs, rcond=None)[0]

    # make it symmetric positive definite
    perm = nearest_spd(perm.reshape(2, 2)).ravel()

    return perm

In [ ]:
import multiprocessing

def process_subdomain(sub_sd_id, sub_sd, perm_dict, folder_results, part, kxx_up, kxy_up, kyx_up, kyy_up):
    mask = part == sub_sd_id
    sub_perm = {key: val[mask] for key, val in perm_dict.items()}

    folder_x = folder_results + str(sub_sd_id) + "_x"
    q_h, grad_h = upscale(sub_sd, sub_perm, 0, folder_x)

    folder_y = folder_results + str(sub_sd_id) + "_y"
    q_v, grad_v = upscale(sub_sd, sub_perm, 1, folder_y)

    kk = compute_tensor(grad_h, grad_v, q_h, q_v)

    return [kk[0], kk[1], kk[2], kk[3]]

def Checkpoint1_solution(selected_layers, folder_results):
    spe10 = Spe10(selected_layers)
    perm_folder = spe10_folder + "/perm/"
    spe10.read_perm(perm_folder)
    perm_dict = spe10.perm_as_dict()

    num_part = 20
    part, sub_sds, sd_coarse = coarse_grid(spe10.sd, num_part)

    kxx_up = np.zeros(spe10.sd.num_cells)
    kxy_up = np.zeros(spe10.sd.num_cells)
    kyx_up = np.zeros(spe10.sd.num_cells)
    kyy_up = np.zeros(spe10.sd.num_cells)
    kxx = np.zeros(spe10.sd.num_cells)

    process = multiprocessing.Pool(processes = 8)
    result = process.starmap(process_subdomain, [(sub_sd_id, sub_sd, perm_dict, folder_results, part, kxx_up, kxy_up, kyx_up, kyy_up) for sub_sd_id, sub_sd in enumerate(sub_sds)])
    process.close()
    process.join()

    for sub_sd_id, kk in enumerate(result):
        mask = part == sub_sd_id
        sub_perm = {key: val[mask] for key, val in perm_dict.items()}
        kxx[mask] = sub_perm["kxx"]
        kxx_up[mask], kxy_up[mask], kyx_up[mask], kyy_up[mask] = kk
        print(f"Subdomain {sub_sd_id}: {kk}")

    var_to_save = [
        ("kxx", kxx_up),
        ("kxy", kxy_up),
        ("kyx", kyx_up),
        ("kyy", kyy_up),
        ("fine", kxx)
    ]

    save = pp.Exporter(spe10.sd, "upscaled_k", folder_name=folder_results)
    save.write_vtu(var_to_save)

    write_upscaled_perm("as_tensor", sd_coarse, result, folder_results)

    return sd_coarse, result

In [ ]:
selected_layers = 20
folder_results = main_folder + "results/"

sd_coarse, result = Checkpoint1_solution(selected_layers, folder_results)